# Plotting a Radar Image

---
Radar data are usually structured as a 2D image, which we describe in terms of the two axis of variability: "fast-time" and "slow-time":
* Slow Time: As the radar instrument is driven or flown over a glacier, it sends out electromagnetic pulses ~10,000 times per second (or every $10^{-4}$ seconds). The exact rate at which pulses are transmitted is defined by the instrument's "PRF", or pulse-repeitition frequency. Each column of pixels in a radar image represents a new radar pulse. Thus, the x-axis of the image is called the "slow-time" axis, as the time between adjacent pixels is > ~$10^{-4}$ seconds.
* Fast Time: After a pulse is generated, the radar instrument measures the strength of the electric field over time. The radar has to take lots of measurements of the field strength very quickly to capture the returning wave, because the speed of light in ice is ~168,000,000 m/s (and ~300,000,000 m/s in air). So, it measures the electric field strength about once every 10 nanoseconds ($10^{-8}$ s, or the "sample rate") over a period of about 10 microseconds ($10^{-5}$ s), and stores those values. Because the time between pixels above and below an observation is about 10,000 times shorter than the time between pixels to the left and right, the y axis is called the "fast-time" axis. 

Let's load in some data, calculate the sample rate, and look at an image.

In [1]:
### sys and glob are useful tools for dealing with the local filesystem.
import glob

### numpy and matplotlib are the core analysis packages for manipulating and plotting data arrays
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('/mnt/data01/Code/')
import NDH_Tools as ndh

In [78]:
### Find and read the radar data file
year='2018'
cont='Antarctica'
dateframe='20181010_02'
seg = 11;

data_folder = '/mnt/data01/Data/RadarData/'+cont+'/'+year+'_'+cont+'*/CSARP_standard/'+dateframe+'/'
fns = sorted(glob.glob(data_folder+'Data_'+dateframe[0]+'*.mat'))
fn = fns[seg-1]
fn

'/mnt/data01/Data/RadarData/Antarctica/2018_Antarctica_DC8/CSARP_standard/20181010_02/Data_20181010_02_011.mat'

In [10]:
radar_data = ndh.loadmat(fn);
## Let's see what the structure of this dictionary is:
radar_data.keys()

dict_keys(['Bottom', 'Data', 'Elevation', 'GPS_time', 'Heading', 'Latitude', 'Longitude', 'Pitch', 'Roll', 'Surface', 'Time', 'file_version', 'param_array', 'param_records', 'param_sar'])

<br><br>

We can start by looking at the slow-time axis -- there are several keys that capture information in slow-time:
* `GPS_time`, which tells you about the time associated with each pulse
* `Latitude`, which tells you the latitude of acquisition for each pulse
* `Longitude`, which tells you the longitude of acquisition for each pulse

We can see that the shapes of all of these variables are the same, and we can calculate the pulse-repitition frequency.


In [12]:
print(radar_data['GPS_time'].shape, radar_data['Latitude'].shape, radar_data['Longitude'].shape)
print(radar_data['GPS_time'][0,1]-radar_data['GPS_time'][0,0])

(1, 3335) (1, 3335) (1, 3335)
0.12607908248901367


<br>
Now let's look at the fast-time axis.  

In [17]:
cice = 168000000
depth = radar_data['Time'][0]/2*cice

print(radar_data['Time'][0].shape)
print(radar_data['Time'][1]-radar_data['Time'][0])

(1,)
[2.e-08]


<br>
Now, given the number of samples in fast time and the number of samples in slow time, we can verify that the radar image is the shape that we expect:

In [18]:
print(radar_data['Data'].shape)

(3154, 3335)


<br><br>

So let's go about plotting this data set. As a first step, I often convert the time axes to distance and depth axes. For that, we have to convert Latitude and Longitude to x and y coordinates, and use those to calculate the distance along the line. This is just an application of the Pythagorean Theorem, done below.

In [27]:
xy = ndh.polarstereo_fwd(radar_data['Latitude'][0],radar_data['Longitude'][0])
distance = ndh.distance_vector(xy['x'],xy['y'])

<br><br> 
Then, we need to calculate the "depth" axis. We convert time-after-transmit-pulse (`radar_data['Time']`) to distance below the instrument (the "range") using the speed of light. Because these radar data were collected from a plane, the light propagates through both air and ice, which have different speeds of light. While it is imperfect, we will calculate the range using only the speed of light in ice. When plotted, the y axis generated can be used to interpret ice thickness this way.

In [72]:
depthdata = ndh.elevation_shift(radar_data['Data'],radar_data['Time'],radar_data['Surface'][0],radar_data['Elevation'][0],radar_data['Bottom'][0])

<br><br><br>
Finally, we can produce the image. The radar data represents an "amplitude" image -- a measure of the peak-to-trough variability in the strength of the electromagnetic field. Because there are exponential losses in amplitude with wave propagation, it is often better to look at the data on a logarithmic scale. So, we oven square the data (converting from "amplitude" to "power") and take the log_10 (which changes our units to decibels, dB).


In [43]:
%matplotlib ipympl

In [71]:
fig = plt.figure(figsize=(15,7))
imdata = plt.imshow(np.log10(depthdata['new_data']**2),extent=[distance[0]/1000,distance[-1]/1000,depthdata['depth_axis'][0],depthdata['depth_axis'][-1]],origin='lower',aspect='auto',cmap='gray',vmin=-35,vmax=-5)
ax = plt.gca()
ax.invert_yaxis()

plt.xlabel('Distance (km)')
plt.ylabel('Elevation (m)')
fig.colorbar(imdata)

markers = np.round(np.linspace(0,len(distance)-1,6)).astype('int')

for i in markers:
    plt.axvline(distance[i]/1000,c='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
import rioxarray as rxr
hs = rxr.open_rasterio('/mnt/data01/Data/Antarctic_Imagery/REMA_200m_hillshade.nc')

In [69]:
spacing = 50000

xs = [xy['x'].min()-spacing,xy['x'].max()+spacing]
ys = [xy['y'].min()-spacing,xy['y'].max()+spacing]

subset_slices = {'x':slice(xs[0],xs[1]),'y':slice(ys[1],ys[0])}

hs_sub = hs.sel(subset_slices)

plt.figure()
hs_sub[0,:,:].plot.imshow(cmap='gray')
plt.plot(xy['x'],xy['y'],c='blue')

plt.plot(xy['x'][markers],xy['y'][markers],'o',c='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …